# 2016 Georgia Election Comparison - Data Cleaning and Exploratory Analysis

Alexander Church

DS 4003

### Goal of project

The goal of the dashboard is to compare the results in congressional districts in Georgia in House elections to presidential elections.

### Data provenence

The data comes from the Election Data and Science Lab at the Massachusetts Institute of Technology. The data comes from two files, [2016-precinct-president.csv](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/LYWX3D) and [2016-precinct-house.tab](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/PSKDUJ).

### Description of data

The data describes election results in the 2016 federal election, for the US House race and the US Presidential race (between Donald Trump and Hillary Clintion). The data was filtered to just cover the state of Georgia, as the entire country would be unfeasible for the project. The data has information by precinct, as well as details such as the method of voting (same-day, early, mail, etc.) which can lead to interesting analysis.

## Data cleaning:

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.express as px
from dash import Dash, html, dcc 

In [2]:
# Import presidential data
pres2016 = pd.read_csv('2016-precinct-president.csv', encoding='ISO 8859-1')

C:\Users\achur\AppData\Local\Temp\ipykernel_11824\46099483.py:2: DtypeWarning: Columns (13,17,19,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  pres2016 = pd.read_csv('2016-precinct-president.csv', encoding='ISO 8859-1')


In [3]:
# Drop needless columns
columns_to_drop = ['stage', 'state_fips','state_icpsr','county_fips',  'county_lat','county_long','office','candidate_last', 'candidate_first', 'candidate_middle', 'candidate_full', 'candidate_suffix', 'candidate_nickname','candidate_google', 'candidate_govtrack', 'candidate_icpsr', 'candidate_maplight', 'candidate_opensecrets', 'candidate_wikidata', 'candidate_fec', 'candidate_party']
pres2016 = pres2016.drop(columns = columns_to_drop)

# Filter out write-in candidates
pres2016 = pres2016[pres2016['writein'] == False]

# Create precinct ID
pres2016['precinctid'] = pres2016['precinct'].astype('string') + pres2016['jurisdiction'].astype('string') + pres2016['state_postal']

# Filter to Georgia
pres2016 = pres2016[pres2016['state'] == 'Georgia']

# Reset index
pres2016 = pres2016.reset_index(drop=True)

# New column for 'race' - which election
pres2016['race'] = 'pres'

In [4]:
# Import house data
house2016 = pd.read_csv('2016-precinct-house.tab', encoding='ISO 8859-1')

C:\Users\achur\AppData\Local\Temp\ipykernel_11824\621711366.py:2: DtypeWarning: Columns (29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  house2016 = pd.read_csv('2016-precinct-house.tab', encoding='ISO 8859-1')


In [5]:
# Filter to Georgia
house2016 = house2016[house2016['state'] == 'Georgia']

# Drop needless columns
house2016 = house2016.drop(columns = columns_to_drop)

# Filter out write-in candidates
house2016 = house2016[house2016['writein'] == False]

# Create precinct ID
house2016['precinctid'] = house2016['precinct'] + house2016['jurisdiction'] + house2016['state_postal']

# New column for 'race' - which election
house2016['race'] = 'house'

# Exclude certain type of data
house2016 = house2016[house2016['mode'] != 'total']

# Reset index
house2016 = house2016.reset_index(drop=True)

# Clean certain candidates
house2016 = house2016.replace('EARL â\x80\x9cBUDDYâ\x80\x9d CARTER', 'Earl Carter')
house2016 = house2016.replace('HENRY C â\x80\x9cHANKâ\x80\x9d JOHNSON JR', 'Henry Johnson Jr.')

In [6]:
#Combine dataframes, reset index, drop needless columns
all2016 = pd.concat([pres2016, house2016]).reset_index(drop = True).drop(
    columns = ['special', 'state_postal', 'county_ansi', 'candidate_normalized', 'writein', 'candidate_fec_name'])

all2016

,year,state,county_name,jurisdiction,precinct,candidate,district,party,mode,votes,precinctid,race
0,2016,Georgia,Appling County,Appling,2,Hillary Clinton,statewide,democratic,election day,338,2ApplingGA,pres
1,2016,Georgia,Appling County,Appling,2,Hillary Clinton,statewide,democratic,absentee by mail,87,2ApplingGA,pres
2,2016,Georgia,Appling County,Appling,2,Hillary Clinton,statewide,democratic,advance in person,317,2ApplingGA,pres
3,2016,Georgia,Appling County,Appling,2,Hillary Clinton,statewide,democratic,provisional,0,2ApplingGA,pres
4,2016,Georgia,Appling County,Appling,2,Gary Johnson,statewide,libertarian,election day,5,2ApplingGA,pres
...,...,...,...,...,...,...,...,...,...,...,...,...
50287,2016,Georgia,Worth County,Worth,Warwick,JAMES NEAL HARRIS,8.0,democratic,provisional,0,WarwickWorthGA,house
50288,2016,Georgia,Worth County,Worth,Warwick,AUSTIN SCOTT,8.0,republican,election day,212,WarwickWorthGA,house
50289,2016,Georgia,Worth County,Worth,Warwick,AUSTIN SCOTT,8.0,republican,absentee by mail,13,WarwickWorthGA,house
50290,2016,Georgia,Worth County,Worth,Warwick,AUSTIN SCOTT,8.0,republican,advance in person,72,WarwickWorthGA,house


## Data dictionary:

| Variable | Full Name | Description |
|----------|-----------|-------------|
|year|Year|Year of Election (always 2016)|
|state|State|State of Election (always Georgia)|
|county_name|County Name|Name of County of Election|
|jurisdiction|Jurisdiction|Jurisdiction running election (always name of county)|
|precinct|Precinct|Precinct of election|
|candidate|Candidate|Name of candidate|
|district|District|House district of precinct ('statewide' for presidential rows)|
|party|Political Party|Political party of candidate|
|mode|Voting Mode|Mode of Voting (election day, mail, etc.)|
|votes|Votes|Number of votes in precinct|
|precinctid|Precinct ID|Unique ID for Precinct, precinct name concatenated with jurisdiction|
|race|Race|Election candidate is running in (house or pres)|

## Exploratory data analysis:

### Summary of data

In [7]:
## Precinct counts

precinctshouse = all2016[all2016['race'] == 'house']['precinctid'].unique() # House precincts
precinctspres = all2016[all2016['race'] == 'pres']['precinctid'].unique() # Pres precincts

print(len(precinctshouse))
print(len(precinctspres))

# Summary of dataset
print(all2016.head())

2693
2693
   year    state     county_name jurisdiction precinct        candidate  \
0  2016  Georgia  Appling County      Appling        2  Hillary Clinton   
1  2016  Georgia  Appling County      Appling        2  Hillary Clinton   
2  2016  Georgia  Appling County      Appling        2  Hillary Clinton   
3  2016  Georgia  Appling County      Appling        2  Hillary Clinton   
4  2016  Georgia  Appling County      Appling        2     Gary Johnson   

    district        party               mode  votes  precinctid  race  
0  statewide   democratic       election day    338  2ApplingGA  pres  
1  statewide   democratic   absentee by mail     87  2ApplingGA  pres  
2  statewide   democratic  advance in person    317  2ApplingGA  pres  
3  statewide   democratic        provisional      0  2ApplingGA  pres  
4  statewide  libertarian       election day      5  2ApplingGA  pres  


The data shows 2,693 precincts in Georgia for the House election and the presidential election. Georgia has 159 counties and 14 districts, so this makes approximately 16.9 precincts per county and 192 precincts per congressional district.

The only continuous variable in the dataset is votes. Therefore, all representations of the data will be filtered and divided, but only have one output variable (votes). This limits the amount of representations, but forces the dashboard to be creative.

### Vote counts per race

In [8]:
# Filter presidential data
presrace = all2016[all2016['district'] == 'statewide']

# Filter house data
houseraces = all2016[all2016['race'] == 'house']

# Create grouped bar graph
fig1 = px.histogram(houseraces, 
                   x = "district",  # by district
                   y = "votes", # total votes per party
                   color = 'party', # republican or democratic
                   barmode = 'group',
                   histfunc = 'sum', # sum of votecount
                   title = 'Results in Georgia Congressional Districts',
                   color_discrete_map = {"republican": "red", "democratic": "blue"}, # matching colors
                   height=400)

fig1.show() # Display figure

The results are most interesting in the four districts where a democratic candidate was not fielded (1, 9, 10, and 14). In District 1, the lack of an opponent likely reduced turnout dramatically, whereas in Districts 9, 10, and 14, this did not have a large effect on turnount.

### Party votes by type

In [9]:
fig2 = px.histogram(
            all2016[all2016['mode'] != 'total'][all2016['mode'] != 'provisional'][all2016['party'] != 'libertarian'], # Dataframe filtering
             x="party", # by party
             y="votes",
             color="party",
             facet_col="mode", # seperate by mode of voting
             histfunc = 'sum', # sum of votecount
             title = 'Results by Voting Type',
             color_discrete_map = {"republican": "red", "democratic": "blue"}, # matching colors
             category_orders={'mode': ['election day', 'absentee by mail', 'advance in person'], "race": ["pres", "house"]})

fig2.show()

C:\Users\achur\AppData\Local\Temp\ipykernel_11824\138672627.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



Interestingly, the party split roughly stays the same between different modes of voting. Specifically after the politicization of mail-in voting in the 2020 election, it is expected that more Democrats engage in mail-in voting than Republicans; however, that assumption does not hold for this election. There was also much less mail-in voting in 2016 than we now expect, mostly due to the COVID-19 pandemic.

### Possible List of UI Components

- Dropdown to filter between House districts
- Radio buttons to filter House vs President
- Checkboxes to filter voting mode
- Dropdown to filter between candidates
- With more data (which I may organize and include), slider from 2016 to 2020

### Possible Data Visualizations

- Geographic representation of results, with districts shaded different colors
    - Can be filtered for two different elections, or side-by-side
- Individual district results broken down by:
    - Voting mode
    - Comparison between house results and Presidential results
    - With more data (which I may organizae and include), trends from 2016 to 2020

### Export to CSV (commented out):

In [10]:
all2016.to_csv('data.csv', index=False)